In [1]:
%reload_ext autoreload
%autoreload 2
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
from utils import *

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

In [2]:
print("Available devices: ", torch.cuda.device_count())
print("torch version:", torch.__version__)
print("cudnn version:", torch.backends.cudnn.version())
print("cuda version:", torch.version.cuda)

Available devices:  2
torch version: 2.2.1+cu121
cudnn version: 8902
cuda version: 12.1


In [3]:

max_length = 2048
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [4]:
#Convert midi into dtype int
midi_train = midi_train.astype(int)
midi_test = midi_test.astype(int)

In [5]:
print(train.shape, test.shape, midi_train.shape, midi_test.shape)

dataset = TokenDatasetMidi(train, midi_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_test, max_length, tokens)

(43272, 2048) (4800, 2048) (43272, 2048, 8) (4800, 2048, 8)
data has 43272 pieces, 195 unique tokens.
data has 4800 pieces, 195 unique tokens.


In [6]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 250,
    }
)

03/13/2024 17:46:26 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


In [8]:
epochs = 100
embedding = 192
heads = 4
layers = 4
batch_size = 32
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()

03/13/2024 17:47:09 - INFO - model -   number of parameters: 1.861376e+06
03/13/2024 17:47:10 - INFO - model -   number of parameters: 1.861376e+06


../models/model_epochs->100_heads->4_embd->192_batch->32_new_midi_embeddings


epoch 1 iter 1352: train loss 1.67072. lr 3.000000e-05: 100%|██████████| 1353/1353 [05:18<00:00,  4.25it/s]
03/13/2024 17:52:28 - INFO - trainer -   epoch train loss: 2.749363


train loss: 2.7493631558689646


03/13/2024 17:52:42 - INFO - trainer -   test loss: 1.633470
epoch 2 iter 1352: train loss 1.45663. lr 3.000000e-05: 100%|██████████| 1353/1353 [05:18<00:00,  4.24it/s]
03/13/2024 17:58:02 - INFO - trainer -   epoch train loss: 1.582591


...

train loss: 0.16990903396564153


03/14/2024 03:01:26 - INFO - trainer -   test loss: 0.163983


Model ../models/model_epochs->100_heads->4_embd->192_batch->32_new_midi_embeddings saved


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train loss,█▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,█▇▆▆▅▅▅▅▅▅▅▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
hits@1,▁▂▃▃▃▃▃▃▄▄▄▄▅▅▆▇▇▇▇▇████████████████████
hits@3,▁▃▅▅▅▅▆▆▆▆▆▆▆▇██████████████████████████
hits@5,▁▂▅▅▆▆▆▆▇▇▇▇▇▇██████████████████████████
Train loss,0.16991
Validation loss,0.16398
hits@1,0.95847
hits@3,0.98835
hits@5,0.99313
